In [13]:
# Intalar pymupdf
# %pip install pymupdf

#importar fitz para usar pymupdf (fitz es pymupdf)
import fitz  

In [14]:
# Instalar e importar la biblioteca pendulum para trabajar con fechas
# %pip install pendulum

import pendulum

# Definir timezone e idioma en la biblioteca pendulum

tz = pendulum.timezone('Europe/Paris')
pendulum.set_locale('es')

In [29]:
# Importar regex
import re

In [26]:
# Intalar e importar la biblioteca para trabajar con codigos postales
# %pip install pgeocode

import pgeocode

In [15]:
def formato_fecha(fecha):
    dt = ''
    if '/' in fecha:
        dt = pendulum.from_format(fecha, 'DD/MM/YYYY')
    elif 'de' in fecha:
        dt = pendulum.from_format(fecha.replace(' de', ''), 'DD MMMM YYYY')
    try:
        dt_final = dt.format('DD.MM.YYYY')
    except Exception:
        dt_final = 'fecha'  
    return dt_final    


In [16]:
# Extraer el texto de cada página como string en una lista 
file = fitz.open("pdfs/factura_915.pdf")
pymupdf_text = []

for page in file:
    pymupdf_text.append(page.get_text())

print(pymupdf_text)    

[' \n \n \nDATOS DE LA FACTURA \nIMPORTE FACTURA: 135,27 € \nNº factura: UN3212861143 \nReferencia: 510006173251/4418 \nPeriodo de consumo: 02/04/2021 a 01/06/2021 \n \nFecha de cargo: 06 de junio de 2021 \nENERCOLUZ ENERGIA SL \nCif: B40178550 \nAVDA. DE LOS TOROS, 7 40200 - CUELLAR \nRaimundo Ulloa Perdomo \nTravesia San Andrés \n25267 Sant Martí de Riucorb  \nLleida \nPor potencia contratada \nPor energía consumida \nImpuesto electricidad \nAlquiler equipos de medida y control \n \nIGIC reducido ( 15%) \n37,68 € \n72,77 € \n5,65 € \n1,60 € \n17,41 € \nIGIC normal \n( 10%) \n0,16 € \n........................................................................................................................................... \nTOTAL IMPORTE FACTURA \n135,27 € \nINFORMACIÓN DEL CONSUMO ELÉCTRICO \nConsumo en el \nperiodo llano \nDe 0h a 24h \n....................................................................................................................................................

In [17]:
""" En caso de necesitar el texto de todas las páginas del pdf 
lineas_pdf = []
for x in range(len(pymupdf_text)):
    lineas = pymupdf_text[x].split('\n')
    lineas_pdf.append(lineas)
lineas_pdf 
""" 

" En caso de necesitar el texto de todas las páginas del pdf \nlineas_pdf = []\nfor x in range(len(pymupdf_text)):\n    lineas = pymupdf_text[x].split('\n')\n    lineas_pdf.append(lineas)\nlineas_pdf \n"

In [18]:
# Crear el diccionario que se convertirá en el archivo JSON final con los datos a conseguir
diccionario_final = {
    "nombre_cliente": "",
    "dni_cliente": "",
    "calle_cliente": "",
    "cp_cliente": "",
    "población_cliente": "",
    "provincia_cliente": "",
    "nombre_comercializadora": "",
    "cif_comercializadora": "",
    "dirección_comercializadora": "",
    "cp_comercializadora": "",
    "población_comercializadora": "",
    "provincia_comercializadora": "",
    "número_factura": "",
    "inicio_periodo": "",
    "fin_periodo": "",
    "importe_factura": "",
    "fecha_cargo": "",
    "consumo_periodo": "",
    "potencia_contratada": ""
}

In [19]:
# Extraer el texto de la primera página del pdf (dónde está toda la información necesaria)
lineas_pag1 = pymupdf_text[0].split('\n')

In [20]:
# Se limpian los datos para eliminar las líneas vacias.
# Se crea un diccionario con los apartados del pdf del que se pueden extraer pares key/valor mediante la separación por los ':'.
# Se crea una lista con todos los datos para tenerlos ordenados.

dict_lineas = {}
lista_lineas = []
contador = 0
for linea in lineas_pag1:
    linea = linea.strip()
    #Se eliminan lineas vacias (o de solo un espacio)
    if linea != '' and linea != ' ':
        lista_lineas.append(linea)
        #Se dividen las líneas por el primer ':' para separarlas en Key/valor en el diccionario
        if ':' in linea:
            lista_linea = linea.split(':', 1)
            dict_lineas[lista_linea[0]] = lista_linea[1].strip() 
        else:
            # En caso de no contar con ':' en la línea, se añade al diccionario con la key?otro' más un número incremental
            contador += 1
            dict_lineas[f'otro{contador}'] = linea

dict_lineas         

{'otro1': 'DATOS DE LA FACTURA',
 'IMPORTE FACTURA': '135,27 €',
 'Nº factura': 'UN3212861143',
 'Referencia': '510006173251/4418',
 'Periodo de consumo': '02/04/2021 a 01/06/2021',
 'Fecha de cargo': '06 de junio de 2021',
 'otro2': 'ENERCOLUZ ENERGIA SL',
 'Cif': 'B40178550',
 'otro3': 'AVDA. DE LOS TOROS, 7 40200 - CUELLAR',
 'otro4': 'Raimundo Ulloa Perdomo',
 'otro5': 'Travesia San Andrés',
 'otro6': '25267 Sant Martí de Riucorb',
 'otro7': 'Lleida',
 'otro8': 'Por potencia contratada',
 'otro9': 'Por energía consumida',
 'otro10': 'Impuesto electricidad',
 'otro11': 'Alquiler equipos de medida y control',
 'otro12': 'IGIC reducido ( 15%)',
 'otro13': '37,68 €',
 'otro14': '72,77 €',
 'otro15': '5,65 €',
 'otro16': '1,60 €',
 'otro17': '17,41 €',
 'otro18': 'IGIC normal',
 'otro19': '( 10%)',
 'otro20': '0,16 €',
 'otro21': '...........................................................................................................................................',
 'otro22': 'TOTA

In [21]:
lista_lineas

['DATOS DE LA FACTURA',
 'IMPORTE FACTURA: 135,27 €',
 'Nº factura: UN3212861143',
 'Referencia: 510006173251/4418',
 'Periodo de consumo: 02/04/2021 a 01/06/2021',
 'Fecha de cargo: 06 de junio de 2021',
 'ENERCOLUZ ENERGIA SL',
 'Cif: B40178550',
 'AVDA. DE LOS TOROS, 7 40200 - CUELLAR',
 'Raimundo Ulloa Perdomo',
 'Travesia San Andrés',
 '25267 Sant Martí de Riucorb',
 'Lleida',
 'Por potencia contratada',
 'Por energía consumida',
 'Impuesto electricidad',
 'Alquiler equipos de medida y control',
 'IGIC reducido ( 15%)',
 '37,68 €',
 '72,77 €',
 '5,65 €',
 '1,60 €',
 '17,41 €',
 'IGIC normal',
 '( 10%)',
 '0,16 €',
 '...........................................................................................................................................',
 'TOTAL IMPORTE FACTURA',
 '135,27 €',
 'INFORMACIÓN DEL CONSUMO ELÉCTRICO',
 'Consumo en el',
 'periodo llano',
 'De 0h a 24h',
 '..................................................................................................

In [22]:
# Se obtienen todos los datos que se pueden extraer mediante las llaves del diccionario 
for key in dict_lineas.keys():
    if 'importe' in key.lower():
        diccionario_final["importe_factura"] = dict_lineas[key].split()[0]
    if 'nº' in key.lower():
        diccionario_final["número_factura"] = dict_lineas[key]
    if 'periodo' in key.lower():
        fechas_consumo = dict_lineas[key].split('a')
        diccionario_final["inicio_periodo"] = formato_fecha(fechas_consumo[0].strip())
        diccionario_final["fin_periodo"] = formato_fecha(fechas_consumo[-1].strip())
    if 'fecha' in key.lower() and 'cargo' in key.lower():
        diccionario_final["fecha_cargo"] = formato_fecha(dict_lineas[key])
    if 'potencia' in key.lower():
        diccionario_final["potencia_contratada"] = dict_lineas[key].split()[0]     
    if 'titular' in key.lower():
        diccionario_final["nombre_cliente"] = dict_lineas[key]     
    if 'domicilio social' in key.lower():
        diccionario_final["provincia_comercializadora"] = dict_lineas[key].split('-')[-1].strip()   

# Se duplica el for en el caso del cif y el Nif para poder romper el loop después de la primera localización y asegurarnos que es el de la comercializadora y el cliente respectivamente

for key in dict_lineas.keys():
    if 'cif' in key.lower():
        diccionario_final["cif_comercializadora"] = dict_lineas[key]  
        break 

for key in dict_lineas.keys():
    if 'nif' in key.lower():
        diccionario_final["dni_cliente"] = dict_lineas[key]  
        break 



In [27]:
# Mediante un enumerate extraemos todos los datos que sabemos que estan cerca unos de otros en la lista.
lista_consumo = []
lista_direccion = []
for indice, linea in enumerate(lista_lineas):
    if linea.lower() == 'consumo':
        lista_consumo.append(linea)
        lista_consumo.append(lista_lineas[indice+1]) 
        lista_consumo.append(lista_lineas[indice+2])

        for elemento in lista_consumo:
            if 'kwh' in elemento.lower():
                try:
                    diccionario_final["consumo_periodo"] = int(elemento.split()[0])
                except Exception:
                    continue    
    if 'dirección de suministro' in linea.lower():
        try:
            lista_direccion.append(linea) 
            lista_direccion.append(lista_lineas[indice+1]) 
            direccion_suministro = ' '.join(lista_direccion)
            direccion_dividir = direccion_suministro.split(':', 1)
            direccion = direccion_dividir[1].strip().split(',')
            diccionario_final["calle_cliente"] = direccion[0].strip() 
            diccionario_final["población_cliente"] = direccion[1].strip()  
        except Exception:
            direccion_entera = linea.split(':', 1)[1]
            direccion = re.split(r"(\d{5})", direccion_entera)
            diccionario_final["calle_cliente"] = direccion[0].strip() 
            diccionario_final["población_cliente"] = direccion[2].strip()  
            
# Se duplican los bucles for para poder romperlos después de la primera aparición en aquella información repetida a lo largo del pdf y que puede dar problemas.

for indice, linea in enumerate(lista_lineas):
    if diccionario_final["población_cliente"] in linea and len(linea) > len(diccionario_final["población_cliente"]):
        cp = linea.split()[0]
        try:
            cp_cliente = str(int(cp.strip()))
        except Exception:
            continue    
        diccionario_final["cp_cliente"] = cp_cliente
        break

nomi = pgeocode.Nominatim('es')
diccionario_final["provincia_cliente"] = nomi.query_postal_code(cp_cliente)['county_name']

for indice, linea in enumerate(lista_lineas):
    if diccionario_final["cif_comercializadora"] in linea:
        diccionario_final["nombre_comercializadora"] = lista_lineas[indice-1]
        direccion_comercializadora = lista_lineas[indice+1].split('-')
        
        diccionario_final["dirección_comercializadora"] = direccion_comercializadora[0].strip()[0:-5].strip()
        diccionario_final["población_comercializadora"] = direccion_comercializadora[-1].strip()
        diccionario_final["cp_comercializadora"] = direccion_comercializadora[0].strip()[-5:]
        break     
               

In [28]:
diccionario_final 

{'nombre_cliente': 'Raimundo Ulloa Perdomo',
 'dni_cliente': '53056122K',
 'calle_cliente': 'Travesia San Andrés',
 'cp_cliente': '25267',
 'población_cliente': 'Sant Martí de Riucorb',
 'provincia_cliente': 'Lleida',
 'nombre_comercializadora': 'ENERCOLUZ ENERGIA SL',
 'cif_comercializadora': 'B40178550',
 'dirección_comercializadora': 'AVDA. DE LOS TOROS, 7',
 'cp_comercializadora': '40200',
 'población_comercializadora': 'CUELLAR',
 'provincia_comercializadora': 'Segovia',
 'número_factura': 'UN3212861143',
 'inicio_periodo': '02.04.2021',
 'fin_periodo': '01.06.2021',
 'importe_factura': '135,27',
 'fecha_cargo': '06.06.2021',
 'consumo_periodo': 280,
 'potencia_contratada': '5,610'}

In [25]:
import json
with open("factura_915_meu.json", "w") as fp:
    json.dump(diccionario_final, fp, ensure_ascii=False)